# Notebook 01 : Agrégation des données  
## Home Credit Default Risk

## Contexte du projet

**Problématique métier :**  
Prédire le risque de défaut de paiement pour les clients de Home Credit afin d'optimiser l'octroi de crédit et de minimiser les pertes financières.

**Objectif de ce notebook :**  
Enrichir la table principale `application_train.csv` en agrégeant l'historique comportemental des clients provenant de 6 tables secondaires.  
Cette étape est cruciale car les données historiques sont fortement prédictives du comportement futur de remboursement.

---

## Schéma de la base de données

![Schéma des tables](./DB.png)

**Description des tables :**

1. **application_train.csv** : Table principale contenant les informations statiques sur chaque demande de crédit (307 511 clients)
   - Note: Pour ce projet académique, nous utilisons SEULEMENT application_train.csv (pas de test.csv)  
   - Informations démographiques (âge, genre, situation familiale)  
   - Informations professionnelles (revenus, type d'emploi, ancienneté)  
   - Détails du crédit demandé (montant, annuité, prix du bien)

2. **bureau.csv** : Tous les crédits antérieurs du client auprès d'autres institutions financières rapportés au bureau de crédit (1 716 428 lignes)  
   - Une ligne par crédit externe du client  
   - Informations sur le montant, le statut (actif/fermé), les retards éventuels

3. **bureau_balance.csv** : Soldes mensuels des crédits bureau (27 299 925 lignes)  
   - Historique mois par mois du statut de chaque crédit bureau  
   - Permet de détecter les patterns de retard de paiement

4. **previous_application.csv** : Toutes les demandes de crédit antérieures du client chez Home Credit (1 670 214 lignes)  
   - Une ligne par demande passée  
   - Informations sur le statut (approuvée/refusée), les montants, les conditions

5. **POS_CASH_balance.csv** : Soldes mensuels des prêts POS et Cash antérieurs chez Home Credit (10 001 358 lignes)  
   - Historique mois par mois des crédits POS/Cash  
   - Indicateurs de retard (DPD : Days Past Due)

6. **installments_payments.csv** : Historique de remboursement des échéances pour les crédits Home Credit antérieurs (13 605 401 lignes)  
   - Une ligne par échéance (prévue ou réalisée)  
   - Permet de calculer précisément les retards et sous-paiements

7. **credit_card_balance.csv** : Soldes mensuels des cartes de crédit antérieures chez Home Credit (3 840 312 lignes)  
   - Historique mois par mois des cartes de crédit  
   - Informations sur l'utilisation du crédit, les retraits, les retards

**Total : environ 58 millions de lignes à agréger par client**

---

## Méthodologie d'agrégation

**Approche générale :**  
Pour chaque table secondaire, nous appliquons le processus suivant :

1. **Groupby** par `SK_ID_CURR` (identifiant client unique)  
2. **Agrégations statistiques** : min, max, mean, sum, var selon la nature de la variable  
3. **Merge** avec la table `application_train` sur la clé `SK_ID_CURR`

**Agrégations spécifiques :**

- **Bureau** : Séparation des crédits actifs vs fermés  
- **Previous** : Séparation des demandes approuvées vs refusées  
- **Installments** : Calcul des indicateurs de retard (DPD) et paiement anticipé (DBD)

**Feature engineering :**  
En plus des agrégations simples, nous créons des ratios métier pertinents :

- Ratio revenu/crédit (capacité de remboursement)  
- Ratio annuité/revenu (taux d'endettement)  
- Taux de retard de paiement  
- Différence entre montant payé et attendu  

---

## Sources et Références

Ce notebook s'appuie sur les meilleures pratiques de la communauté Kaggle pour le challenge Home Credit Default Risk.

### Kernel Principal de Référence

**jsaguiar - "LightGBM with Simple Features"**  
- **URL :** https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features  
- **Performance :** Top 2% du challenge (Score AUC : 0.792)  
- **Votes :** 1000+ upvotes sur Kaggle  

**Éléments réutilisés de ce kernel :**

- Structure d'agrégation hiérarchique (bureau → previous → POS/CC)  
- Fonction `one_hot_encoder()`  
- Fonctions d'agrégation :  
  `bureau_and_balance()`, `previous_applications()`, `pos_cash()`,  
  `installments_payments()`, `credit_card_balance()`

**Adaptations et améliorations apportées :**

- Documentation complète : Docstrings détaillées en français  
- Explications pédagogiques : Sections narratives expliquant le "pourquoi"  
- Contexte métier : Impact des features  
- Réorganisation : 24 cellules claires vs 48 originales  
- Commentaires détaillés : Code commenté ligne par ligne  
- Adaptation locale : Chemins et configurations adaptés aux données locales  

### Autres Références

**Will Koehrsen - "Introduction to Manual Feature Engineering"**  
Utilisé pour les concepts de feature engineering et la création de ratios métier.

---

## Note sur la Réutilisation de Code

Cette approche de réutilisation documentée est conforme aux bonnes pratiques académiques et professionnelles en Data Science :

1. Étude des références : Analyse des kernels Kaggle les plus performants  
2. Réutilisation transparente : Code adapté avec citation claire des sources originales  
3. Valeur ajoutée : Documentation extensive, explications métier, adaptation au projet  
4. Transparence totale : Toutes les sources sont clairement identifiées et créditées  

Documentation complète des sources : voir `SOURCES_ET_REFERENCES.txt`.

---

## Résultat attendu

- **Colonnes initiales** : 122 (table application seule)  
- **Colonnes finales** : environ 250-280 (après agrégation des 7 tables)  
- **Nouvelles features** : environ 130-160 variables comportementales  
- **Durée d'exécution** : 30-45 minutes  
- **Fichier de sortie** : `application_train_AGGREGATED.csv` (environ 250-300 MB)

---


In [ ]:
# Imports des librairies nécessaires

import pandas as pd
import numpy as np
import gc  # Garbage collector pour libérer la mémoire
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')  # Masquer les avertissements non critiques

print('Imports réussis')

### Différence avec le kernel Kaggle original :

- **Version Kaggle (originale) :** Utilise `application_train.csv` ET `application_test.csv`
  - But : Entraîner sur train, prédire sur test, soumettre à Kaggle
  
- **Version Openclassrooms (ce notebook) :** Utilise **SEULEMENT** `application_train.csv`
  - But : Créer notre propre split train/validation dans le Notebook 02
  - Permet d'évaluer le modèle avec des métriques fiables (on a le TARGET)

### Modifications apportées :

1.  Fonction `application_train_test()` renommée en `application_train_only()`
2.  Suppression du chargement de `application_test.csv`
3.  Suppression de la concaténation train+test
4.  Le split train/validation sera fait dans le Notebook 02 avec `train_test_split()`

**Résultat :** Dataset final avec 307,511 clients (au lieu de 356,255) prêt pour le split train/validation.

---


## Configuration des chemins

In [ ]:
from pathlib import Path
import os

# Configuration des chemins d'accès aux fichiers
#
# Structure attendue du dossier data:
# data/
#   ├── application_train.csv
#   ├── application_test.csv
#   ├── bureau.csv
#   ├── bureau_balance.csv
#   ├── previous_application.csv
#   ├── POS_CASH_balance.csv
#   ├── installments_payments.csv
#   └── credit_card_balance.csv

# Detecter automatiquement le bon chemin
current_dir = Path.cwd()
print(f"Repertoire courant: {current_dir}")

# Chercher le dossier data
if (current_dir / 'data').exists():
    DATA_DIR = current_dir / 'data'
elif (current_dir.parent / 'data').exists():
    DATA_DIR = current_dir.parent / 'data'
elif (current_dir / 'PROJET_P7_PREMIUM_AVEC_AGREGATION' / 'data').exists():
    DATA_DIR = current_dir / 'PROJET_P7_PREMIUM_AVEC_AGREGATION' / 'data'
else:
    # Lister les dossiers disponibles
    print("\nDossiers disponibles dans le repertoire courant:")
    for item in current_dir.iterdir():
        if item.is_dir():
            print(f"  - {item.name}")
    
    # Chercher data dans les sous-dossiers
    data_folders = list(current_dir.glob('**/data'))
    if data_folders:
        DATA_DIR = data_folders[0]
        print(f"\nDossier 'data' trouve: {DATA_DIR}")
    else:
        raise FileNotFoundError("Impossible de trouver le dossier 'data'. Verifiez votre emplacement.")

print(f"Dossier data utilise: {DATA_DIR}\n")

FILES = {
    'train': DATA_DIR / 'application_train.csv',
    # 'test': DATA_DIR / 'application_test.csv',  # Non utilise pour projet academique
    'bureau': DATA_DIR / 'bureau.csv',
    'bureau_balance': DATA_DIR / 'bureau_balance.csv',
    'previous': DATA_DIR / 'previous_application.csv',
    'pos': DATA_DIR / 'POS_CASH_balance.csv',
    'installments': DATA_DIR / 'installments_payments.csv',
    'credit_card': DATA_DIR / 'credit_card_balance.csv'
}

# Verification de la presence des fichiers
print('Verification des fichiers de donnees:\n')
all_present = True
for name, path in FILES.items():
    exists = path.exists()
    status = '[OK]' if exists else '[MANQUANT]'
    print(f'{status:12} {name:15} : {path.name}')
    if not exists:
        all_present = False

if not all_present:
    print(f"\nERREUR: Certains fichiers sont manquants dans {DATA_DIR}")
    print("\nFichiers CSV presents dans le dossier data:")
    csv_files = list(DATA_DIR.glob('*.csv'))
    if csv_files:
        for f in sorted(csv_files):
            print(f"  - {f.name}")
    else:
        print("  (aucun fichier CSV trouve)")
else:
    print("\nTous les fichiers requis sont presents !")

## Fonction utilitaire : Encodage des variables catégorielles

**Problématique :**  
Les algorithmes de machine learning (comme LightGBM ou la régression logistique) ne peuvent pas traiter directement les variables catégorielles sous forme de texte. Il faut les transformer en variables numériques.

**Solution : One-Hot Encoding**

Cette technique crée une colonne binaire (0/1) pour chaque modalité de la variable catégorielle.

**Exemple :**

Avant One-Hot Encoding:  
CODE_GENDER  

M  
F  
M  

Après One-Hot Encoding:  
CODE_GENDER_M  CODE_GENDER_F  

1              0  
0              1  
1              0  

**Paramètre `nan_as_category` :**
- Si `True` : crée une colonne supplémentaire pour les valeurs manquantes  
- Si `False` : les NaN restent NaN après l'encodage  

Cette fonction sera utilisée sur toutes les tables pour encoder les variables catégorielles avant agrégation.


In [ ]:
def one_hot_encoder(df, nan_as_category=True):
    """
    Encode les variables catégorielles en variables binaires (One-Hot Encoding)
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète ajoutée, commentaires détaillés
    
    Paramètres:
    -----------
    df : DataFrame
        DataFrame contenant les données à encoder
    nan_as_category : bool, default=True
        Si True, crée une colonne pour les valeurs manquantes (NaN)
    
    Retourne:
    ---------
    df : DataFrame
        DataFrame avec variables catégorielles encodées en binaires
    new_columns : list
        Liste des nouvelles colonnes créées par l'encoding
    """
    
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    
    new_columns = [c for c in df.columns if c not in original_columns]
    
    return df, new_columns


## 1. Traitement de la table APPLICATION et Feature Engineering

### Objectif
Préparer la table principale `application_train.csv` et créer des features métier pertinentes avant d'agréger les autres tables.

### Étapes de traitement

**1. Concaténation train + test**
- Permet d'appliquer les mêmes transformations sur les deux datasets
- La variable `TARGET` (0/1) n'existe que dans le train

**2. Nettoyage des données**
- Suppression des 4 lignes avec `CODE_GENDER = 'XNA'` (valeur aberrante)
- Correction de `DAYS_EMPLOYED = 365243` (code pour 'inconnu') en `NaN`

**3. Encodage des variables binaires**
- `CODE_GENDER` : M/F encodé en 0/1
- `FLAG_OWN_CAR` : Y/N encodé en 0/1
- `FLAG_OWN_REALTY` : Y/N encodé en 0/1

**4. One-Hot Encoding des autres catégorielles**
- Exemple : `NAME_EDUCATION_TYPE` → 5 colonnes binaires (une par niveau d'éducation)

**5. Feature Engineering : Ratios métier**

Ces ratios capturent des relations importantes entre les variables :

| Feature | Formule | Interprétation métier |
|---------|---------|----------------------|
| `DAYS_EMPLOYED_PERC` | `DAYS_EMPLOYED / DAYS_BIRTH` | Stabilité professionnelle : quelle part de sa vie il a travaillé |
| `INCOME_CREDIT_PERC` | `AMT_INCOME_TOTAL / AMT_CREDIT` | Capacité de remboursement : revenu par rapport au crédit demandé |
| `INCOME_PER_PERSON` | `AMT_INCOME_TOTAL / CNT_FAM_MEMBERS` | Revenu disponible par personne du foyer |
| `ANNUITY_INCOME_PERC` | `AMT_ANNUITY / AMT_INCOME_TOTAL` | Taux d'endettement : part du revenu consacrée au crédit |
| `PAYMENT_RATE` | `AMT_ANNUITY / AMT_CREDIT` | Taux de remboursement annuel |

**Pourquoi ces ratios sont-ils importants ?**
- Un ratio `INCOME_CREDIT_PERC` élevé indique une bonne capacité de remboursement
- Un ratio `ANNUITY_INCOME_PERC` trop élevé (>40%) peut indiquer un sur-endettement
- Ces features créées manuellement améliorent significativement les performances du modèle

**Référence :** Ces features sont inspirées des kernels Kaggle gagnants (jsaguiar, tunguz)

In [ ]:
def application_train_only(num_rows=None, nan_as_category=False):
    """
    Charge et prétraite SEULEMENT application_train (version académique)
    
    MODIFICATION: Version adaptée pour projet académique sans application_test.csv
    Le code original (jsaguiar) utilisait train+test pour la compétition Kaggle.
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : 
    - Suppression de l'utilisation de application_test.csv
    - Documentation des features créées
    - Ajout explications ratios métier
    
    Features créées (exemples) :
    - DAYS_EMPLOYED_PERC : Ratio ancienneté emploi / âge
    - INCOME_CREDIT_PERC : Ratio revenu / montant crédit (capacité remboursement)
    - INCOME_PER_PERSON : Revenu par personne dans le foyer
    - ANNUITY_INCOME_PERC : Taux d'endettement (annuité / revenu)
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger (pour tests). None = tout charger
    nan_as_category : bool, default=False
        Traiter les NaN comme une catégorie lors de l'encoding
    
    Retourne:
    ---------
    df : DataFrame
        DataFrame prétraité avec features engineered
    """
    
    print('\n' + '='*60)
    print('TRAITEMENT TABLE APPLICATION (TRAIN SEULEMENT)')
    print('='*60)
    
    # Lecture SEULEMENT des données train
    df = pd.read_csv(FILES['train'], nrows=num_rows)
    print(f"Échantillons train: {len(df)}")
    
    # NOTE: Pas de chargement de test_df ni de concaténation
    # Le split train/validation sera fait dans le notebook 02
    
    # Nettoyage : suppression des 4 lignes avec CODE_GENDER = 'XNA' (valeur aberrante)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Encodage binaire des variables à 2 modalités
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # One-Hot Encoding des variables catégorielles restantes
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Correction valeur aberrante : DAYS_EMPLOYED = 365243 signifie 'inconnu'
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # ==================== FEATURE ENGINEERING ====================
    # Création de ratios métier pertinents pour la prédiction
    
    # 1. Ratio ancienneté emploi / âge (%)
    # Plus ce ratio est élevé, plus la personne a travaillé longtemps relativement à son âge
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    # 2. Ratio revenu / montant crédit (capacité de remboursement)
    # Plus ce ratio est élevé, plus le client a les moyens de rembourser
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    
    # 3. Revenu par personne dans le foyer
    # Permet d'estimer le niveau de vie réel
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    
    # 4. Taux d'endettement (annuité / revenu)
    # Indicateur clé : plus il est élevé, plus le risque est important
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    
    # 5. Taux de paiement (annuité / crédit)
    # Permet d'estimer la durée du crédit
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    # Suppression de la colonne FLAG_DOCUMENT_3 (trop de valeurs manquantes)
    df = df.drop(['FLAG_DOCUMENT_3'], axis=1, errors='ignore')
    
    print(f"Shape finale : {df.shape}")
    print(f"Features créées : DAYS_EMPLOYED_PERC, INCOME_CREDIT_PERC, INCOME_PER_PERSON, etc.")
    
    return df


## 2. Bureau + Bureau_Balance

In [ ]:
def bureau_and_balance(num_rows=None, nan_as_category=True):
    """
    Agrège les données des tables bureau.csv et bureau_balance.csv
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète, explications hiérarchie agrégation
    
    Agrégation hiérarchique en 2 étapes :
    1. bureau_balance → bureau (agrégation par SK_ID_BUREAU, niveau crédit)
    2. bureau → application (agrégation par SK_ID_CURR, niveau client)
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger. None = tout charger
    nan_as_category : bool, default=True
        Traiter les NaN comme une catégorie
    
    Retourne:
    ---------
    bureau_agg : DataFrame
        DataFrame agrégé au niveau client avec statistiques bureau
    """
    
    print('\n' + '='*60)
    print('BUREAU + BUREAU_BALANCE')
    print('='*60)
    
    bureau = pd.read_csv(FILES['bureau'], nrows=num_rows)
    bb = pd.read_csv(FILES['bureau_balance'], nrows=num_rows)
    print(f'Bureau: {bureau.shape}')
    print(f'Bureau_balance: {bb.shape}')
    
    # One-hot encoding
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    print('\n Agrégation bureau_balance...')
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    print('\n Agrégations numériques...')
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ['mean']
    for cat in bb_cat:
        cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits
    print('\n ACTIVE credits...')
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits
    print(' CLOSED credits...')
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    del closed, closed_agg, bureau
    gc.collect()
    
    print(f'\n Bureau aggregated: {bureau_agg.shape}')
    return bureau_agg


## 3. Previous Applications

In [ ]:
def previous_applications(num_rows=None, nan_as_category=True):
    """
    Agrège les données de previous_application.csv
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète, explications logique métier
    
    Analyse les demandes de crédit antérieures chez Home Credit.
    Séparation approved/refused pour capturer patterns différents.
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger. None = tout charger
    nan_as_category : bool, default=True
        Traiter les NaN comme une catégorie
    
    Retourne:
    ---------
    prev_agg : DataFrame
        DataFrame agrégé au niveau client
    """
    
    print('\n' + '='*60)
    print('PREVIOUS APPLICATIONS')
    print('='*60)
    
    prev = pd.read_csv(FILES['previous'], nrows=num_rows)
    print(f'Previous: {prev.shape}')
    
    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)
    
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    
    # FEATURE ENGINEERING
    print('\n Feature Engineering:')
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    print('   APP_CREDIT_PERC (demandé / reçu)')
    
    # Previous applications numeric features
    print('\n Agrégations numériques...')
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Previous Applications: Approved Applications
    print('\n APPROVED applications...')
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications
    print(' REFUSED applications...')
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    
    print(f'\n Previous aggregated: {prev_agg.shape}')
    return prev_agg


## 4. POS_CASH Balance

In [ ]:
def pos_cash(num_rows=None, nan_as_category=True):
    """
    Agrège les données de POS_CASH_balance.csv
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète, explications DPD (Days Past Due)
    
    Analyse l'historique mensuel des crédits POS (point de vente) et Cash.
    Indicateurs de retard (DPD) importants pour prédiction.
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger. None = tout charger
    nan_as_category : bool, default=True
        Traiter les NaN comme une catégorie
    
    Retourne:
    ---------
    pos_agg : DataFrame
        DataFrame agrégé au niveau client
    """
    
    print('\n' + '='*60)
    print('POS_CASH BALANCE')
    print('='*60)
    
    pos = pd.read_csv(FILES['pos'], nrows=num_rows)
    print(f'POS_CASH: {pos.shape}')
    
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    
    # Features
    print('\n Agrégations...')
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    
    print(f'\n POS_CASH aggregated: {pos_agg.shape}')
    return pos_agg


## 5. Installments Payments

In [ ]:
def installments_payments(num_rows=None, nan_as_category=True):
    """
    Agrège les données de installments_payments.csv
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète, explications calculs retards
    
    Analyse l'historique des paiements d'échéances.
    Calculs de retards et sous-paiements = indicateurs forts de risque.
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger. None = tout charger
    nan_as_category : bool, default=True
        Traiter les NaN comme une catégorie
    
    Retourne:
    ---------
    ins_agg : DataFrame
        DataFrame agrégé au niveau client
    """
    
    print('\n' + '='*60)
    print('INSTALLMENTS PAYMENTS')
    print('='*60)
    
    ins = pd.read_csv(FILES['installments'], nrows=num_rows)
    print(f'Installments: {ins.shape}')
    
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    
    # FEATURE ENGINEERING - Payment behavior
    print('\n Feature Engineering:')
    
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    print('   PAYMENT_PERC (payé / prévu)')
    
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    print('   PAYMENT_DIFF (différence paiement)')
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    print('   DPD (Days Past Due - retards)')
    print('   DBD (Days Before Due - paiements anticipés)')
    
    # Features: Perform aggregations
    print('\n Agrégations...')
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    del ins
    gc.collect()
    
    print(f'\n Installments aggregated: {ins_agg.shape}')
    return ins_agg


## 6. Credit Card Balance

In [ ]:
def credit_card_balance(num_rows=None, nan_as_category=True):
    """
    Agrège les données de credit_card_balance.csv
    
    Source : jsaguiar - "LightGBM with Simple Features"
    Adaptations : Documentation complète, explications utilisation crédit
    
    Analyse l'historique mensuel des cartes de crédit.
    Taux d'utilisation du crédit = indicateur comportemental important.
    
    Paramètres:
    -----------
    num_rows : int, optional
        Nombre de lignes à charger. None = tout charger
    nan_as_category : bool, default=True
        Traiter les NaN comme une catégorie
    
    Retourne:
    ---------
    cc_agg : DataFrame
        DataFrame agrégé au niveau client
    """
    
    print('\n' + '='*60)
    print('CREDIT CARD BALANCE')
    print('='*60)
    
    cc = pd.read_csv(FILES['credit_card'], nrows=num_rows)
    print(f'Credit card: {cc.shape}')
    
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # General aggregations
    print('\n Agrégations...')
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    del cc
    gc.collect()
    
    print(f'\n Credit card aggregated: {cc_agg.shape}')
    return cc_agg


## 7. PIPELINE COMPLET - Exécution

### Gestion de la mémoire

Pour éviter les erreurs **MemoryError** lors de l'agrégation de ~58 millions de lignes :

1. **Libération mémoire systématique** : `gc.collect()` après chaque merge
2. **Suppression des variables** : `del variable` pour libérer la RAM immédiatement
3. **Mode debug disponible** : Changez `debug = True` pour tester avec 10,000 lignes

**RAM nécessaire** : ~2-4 GB pour le dataset complet

### Note sur train/test

Contrairement au kernel Kaggle original, nous n'utilisons **QUE** `application_train.csv` (pas de test.csv).

Le split train/validation sera fait dans le Notebook 02 avec `train_test_split()`.

---


In [ ]:
import time
import gc
from pathlib import Path

print('\n' + '='*80)
print(' DEBUT AGGREGATION COMPLETE - 7 TABLES')
print('='*80)

# ============================================================
# CONFIGURATION DES CHEMINS
# ============================================================
# Detecter si on est dans notebooks/ et remonter au niveau parent
current_dir = Path.cwd()
print(f"Repertoire courant: {current_dir}")

# Si on est dans notebooks/, remonter d'un niveau
if current_dir.name == 'notebooks':
    DOSSIER_DATA = current_dir.parent / 'data'
    print(f"Detection: execution depuis notebooks/, remontee au niveau parent")
else:
    DOSSIER_DATA = current_dir / 'data'

print(f"Dossier data utilise: {DOSSIER_DATA.absolute()}")

# Verifier que le dossier existe
if not DOSSIER_DATA.exists():
    print(f"\nERREUR: Le dossier {DOSSIER_DATA.absolute()} n'existe pas !")
    print(f"\nDossiers disponibles dans {current_dir.parent if current_dir.name == 'notebooks' else current_dir}:")
    parent = current_dir.parent if current_dir.name == 'notebooks' else current_dir
    for item in parent.iterdir():
        if item.is_dir():
            print(f"   - {item.name}")
    raise FileNotFoundError(f"Dossier data non trouve: {DOSSIER_DATA}")

# Lister les fichiers disponibles
print(f"\nFichiers CSV disponibles:")
csv_files = list(DOSSIER_DATA.glob('*.csv'))
if not csv_files:
    raise FileNotFoundError(f"ERREUR: Aucun fichier CSV trouve dans {DOSSIER_DATA.absolute()} !")

for f in sorted(csv_files):
    print(f"   - {f.name}")

# Verifier que application_train.csv existe
chemin_train = DOSSIER_DATA / 'application_train.csv'
if not chemin_train.exists():
    print(f"\nERREUR: Le fichier {chemin_train.name} n'existe pas !")
    print(f"   Chemin cherche: {chemin_train.absolute()}")
    print(f"\nSolutions:")
    print(f"   1. Verifiez que les fichiers CSV sont bien dans le dossier 'data/'")
    print(f"   2. Telechargez les donnees depuis Kaggle si necessaire")
    raise FileNotFoundError(f"Fichier requis non trouve: {chemin_train}")

print(f"\nOK - Fichier application_train.csv trouve")

# ============================================================
# AGGREGATION
# ============================================================
start_time = time.time()

# Debug mode (pour tester avec petit echantillon)
debug = False
num_rows = 10000 if debug else None

if debug:
    print(f"\nMODE DEBUG: Chargement de {num_rows} lignes uniquement")
else:
    print(f"\nMODE COMPLET: Chargement de toutes les donnees")

# 1. Application + Feature Engineering
print(f"\n{'='*60}")
print(f"1. TRAITEMENT TABLE APPLICATION")
print(f"{'='*60}")
df = application_train_only(num_rows)
print(f'OK - Shape apres application: {df.shape}')

# 2. Bureau + Bureau_Balance
print(f"\n{'='*60}")
print(f"2. TRAITEMENT BUREAU + BUREAU_BALANCE")
print(f"{'='*60}")
bureau = bureau_and_balance(num_rows)
df = df.join(bureau, how='left', on='SK_ID_CURR')
del bureau
gc.collect()
print(f'OK - Shape apres bureau: {df.shape}')

# 3. Previous Applications
print(f"\n{'='*60}")
print(f"3. TRAITEMENT PREVIOUS APPLICATIONS")
print(f"{'='*60}")
prev = previous_applications(num_rows)
df = df.join(prev, how='left', on='SK_ID_CURR')
del prev
gc.collect()
print(f'OK - Shape apres previous: {df.shape}')

# 4. POS_CASH
print(f"\n{'='*60}")
print(f"4. TRAITEMENT POS_CASH")
print(f"{'='*60}")
pos = pos_cash(num_rows)
df = df.join(pos, how='left', on='SK_ID_CURR')
del pos
gc.collect()
print(f'OK - Shape apres POS_CASH: {df.shape}')

# 5. Installments
print(f"\n{'='*60}")
print(f"5. TRAITEMENT INSTALLMENTS")
print(f"{'='*60}")
ins = installments_payments(num_rows)
df = df.join(ins, how='left', on='SK_ID_CURR')
del ins
gc.collect()
print(f'OK - Shape apres installments: {df.shape}')

# 6. Credit Card
print(f"\n{'='*60}")
print(f"6. TRAITEMENT CREDIT CARD")
print(f"{'='*60}")
cc = credit_card_balance(num_rows)
df = df.join(cc, how='left', on='SK_ID_CURR')
del cc
gc.collect()
print(f'OK - Shape apres credit_card: {df.shape}')

# Note: Pas de separation train/test car nous utilisons SEULEMENT application_train.csv
# Le split train/validation sera fait dans le Notebook 02 avec train_test_split()
train_df = df  # Toutes les donnees sont deja dans train
    
# Liberation memoire finale
del df
gc.collect()
    
elapsed = time.time() - start_time

# ============================================================
# RESUME FINAL
# ============================================================
print('\n' + '='*80)
print(' AGGREGATION TERMINEE AVEC SUCCES')
print('='*80)
print(f'\nRESULTATS FINAUX:')
print(f'   Dataset shape: {train_df.shape}')
print(f'   Nombre de clients: {train_df.shape[0]:,}')
print(f'   Colonnes initiales: 122')
print(f'   Colonnes finales: {train_df.shape[1]}')
print(f'   Nouvelles features: {train_df.shape[1] - 122}')
print(f'\nDuree totale: {elapsed/60:.1f} minutes')

# Verifier qu'on a bien TARGET
if 'TARGET' in train_df.columns:
    print(f'\nDistribution de TARGET:')
    print(f'   Remboursement (0): {(train_df["TARGET"]==0).sum():,} ({(train_df["TARGET"]==0).mean():.1%})')
    print(f'   Defaut (1): {(train_df["TARGET"]==1).sum():,} ({(train_df["TARGET"]==1).mean():.1%})')
else:
    print(f'\nWARNING: Colonne TARGET non trouvee !')

print('\n' + '='*80)

# ============================================================
# SAUVEGARDE
# ============================================================
print(f'\nSauvegarde du fichier agrege...')
chemin_sortie = DOSSIER_DATA / 'application_train_AGGREGATED.csv'
train_df.to_csv(chemin_sortie, index=False)
print(f'OK - Fichier sauvegarde: {chemin_sortie}')
print(f'Taille: {chemin_sortie.stat().st_size / 1e6:.1f} MB')

print('\nPROCESSUS TERMINE !')
print(f'Vous pouvez maintenant executer le Notebook 02 avec {chemin_sortie.name}')

## 8. Sauvegarde

In [ ]:
# Sauvegarder le fichier agrégé
OUTPUT_FILE = DATA_DIR / 'application_train_AGGREGATED.csv'

print(f'\n Sauvegarde en cours...')
train_df.to_csv(OUTPUT_FILE, index=False)

file_size_mb = OUTPUT_FILE.stat().st_size / (1024**2)
print(f'\n Fichier sauvegardé: {OUTPUT_FILE}')
print(f' Taille: {file_size_mb:.1f} MB')
print(f' Shape: {train_df.shape}')

# Statistiques
print(f'\n STATISTIQUES:')
print(f'  NaN total: {train_df.isnull().sum().sum():,}')
print(f'  % NaN: {train_df.isnull().sum().sum() / (train_df.shape[0] * train_df.shape[1]) * 100:.2f}%')

# Exemples de nouvelles features
new_features = [c for c in train_df.columns if any(p in c for p in 
    ['BURO_', 'ACTIVE_', 'CLOSED_', 'PREV_', 'APPROVED_', 'REFUSED_', 
     'POS_', 'INSTAL_', 'CC_', 'PERC', 'RATE', 'RATIO', 'DPD', 'DBD'])]

print(f'\nFEATURES CRÉÉES : {len(new_features)} nouvelles variables')
print(f'  Exemples : {new_features[:10]}')

print('\n' + '='*80)
print('NOTEBOOK 01 TERMINÉ AVEC SUCCÈS')
print('='*80)
print('\nPROCHAINES ÉTAPES :')
print('  1. Notebook 02 : Construction du pipeline de prétraitement sklearn')
print('  2. Notebook 03 : Comparaison des modèles (Dummy, LogReg, LightGBM)')
print('  3. Notebook 04 : Gestion du déséquilibre (SMOTE vs Class Weight)')
print('  4. Notebook 05 : Optimisation du seuil de décision métier')
print('\n' + '='*80)

## Vérification rapide

In [ ]:
# Vérifier qu'on a bien toutes les features attendues
print('\nVÉRIFICATION DES FEATURES PAR TABLE :')
print('\n1. Application (ratios métier) :')
app_features = [c for c in train_df.columns if 'PERC' in c or 'RATE' in c and 'BURO' not in c and 'PREV' not in c]
print(f'   {len(app_features)} features : {app_features[:5]}')

print('\n2. Bureau :')
buro_features = [c for c in train_df.columns if c.startswith('BURO_')]
print(f'   {len(buro_features)} features')

print('\n3. Bureau Active/Closed :')
active_features = [c for c in train_df.columns if c.startswith('ACTIVE_') or c.startswith('CLOSED_')]
print(f'   {len(active_features)} features')

print('\n4. Previous :')
prev_features = [c for c in train_df.columns if c.startswith('PREV_')]
print(f'   {len(prev_features)} features')

print('\n5. Previous Approved/Refused :')
appr_features = [c for c in train_df.columns if c.startswith('APPROVED_') or c.startswith('REFUSED_')]
print(f'   {len(appr_features)} features')

print('\n6. POS_CASH :')
pos_features = [c for c in train_df.columns if c.startswith('POS_')]
print(f'   {len(pos_features)} features')

print('\n7. Installments (avec DPD/DBD) :')
inst_features = [c for c in train_df.columns if c.startswith('INSTAL_')]
dpd_features = [c for c in inst_features if 'DPD' in c or 'DBD' in c]
print(f'   {len(inst_features)} features (dont {len(dpd_features)} DPD/DBD)')

print('\n8. Credit Card :')
cc_features = [c for c in train_df.columns if c.startswith('CC_')]
print(f'   {len(cc_features)} features')

print(f'\nTOTAL NOUVELLES FEATURES : {len(new_features)}')
print(f'COLONNES TOTALES : {train_df.shape[1]}')